In [ ]:
import os
import glob
import shutil

def rename_files_with_offset(dataset_path, output_path):
    """
    Ajoute un offset basé sur le numéro du batch au nom des images et des labels.

    Args:
        dataset_path (str): Chemin du dossier contenant les batches.
        output_path (str): Chemin du dossier où sauvegarder les fichiers renommés.
    """
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_path, exist_ok=True)

    # Créer les sous-dossiers pour images et labels
    images_output_path = os.path.join(output_path, "images")
    labels_output_path = os.path.join(output_path, "labels")
    os.makedirs(images_output_path, exist_ok=True)
    os.makedirs(labels_output_path, exist_ok=True)

    # Récupérer tous les dossiers de batches
    batch_paths = sorted(glob.glob(os.path.join(dataset_path, 'batch*')))

    # Parcourir chaque batch
    for batch_idx, batch_path in enumerate(batch_paths, start=1):
        print(f"Traitement du batch {batch_idx} : {batch_path}")

        # Chemins des dossiers images et labels
        images_path = os.path.join(batch_path, "images")
        labels_path = os.path.join(batch_path, "labels")

        # Vérifier que les dossiers existent
        if not os.path.exists(images_path) or not os.path.exists(labels_path):
            print(f"Dossiers manquants dans {batch_path}, saut du batch.")
            continue

        # Traiter les images
        image_files = glob.glob(os.path.join(images_path, "*.jpg"))
        for image_file in image_files:
            base_name = os.path.basename(image_file)
            # Renommer l'image avec l'offset batch
            new_name = f"batch{batch_idx}_{base_name}"
            new_image_path = os.path.join(images_output_path, new_name)
            print(f"Copie de l'image: {image_file} vers {new_image_path}")
            shutil.copy(image_file, new_image_path)

            # Correspondance du fichier de label
            label_file = os.path.join(labels_path, base_name.replace(".jpg", ".txt"))
            if os.path.exists(label_file):
                new_label_name = f"batch{batch_idx}_{base_name.replace('.jpg', '.txt')}"
                new_label_path = os.path.join(labels_output_path, new_label_name)
                print(f"Copie du label: {label_file} vers {new_label_path}")
                shutil.copy(label_file, new_label_path)
            else:
                print(f"Pas de fichier label correspondant pour {image_file}.")

    print("Renommage terminé avec succès !")

# Exemple d'utilisation
dataset_path = "/kaggle/input/dataset/merged_dataset"  # Chemin vers votre dataset contenant les batches
output_path = "/kaggle/working/renamed_dataset"  # Chemin où sauvegarder les fichiers renommés
rename_files_with_offset(dataset_path, output_path)


In [ ]:
import os
from PIL import Image

def resize_image_and_update_labels(image_path, label_path, output_image_path, output_label_path, new_size):
    # Ouvrir l'image
    with Image.open(image_path) as img:
        # Obtenir les dimensions originales
        original_width, original_height = img.size

        # Redimensionner l'image
        img_resized = img.resize(new_size)
        img_resized.save(output_image_path)

        # Ouvrir le fichier d'annotation
        with open(label_path, 'r') as label_file:
            lines = label_file.readlines()

        # Calculer les facteurs de redimensionnement
        width_ratio = new_size[0] / original_width
        height_ratio = new_size[1] / original_height

        # Mettre à jour les annotations
        with open(output_label_path, 'w') as label_file:
            for line in lines:
                parts = line.strip().split()
                class_id = parts[0]
                x_center = float(parts[1]) * original_width
                y_center = float(parts[2]) * original_height
                width = float(parts[3]) * original_width
                height = float(parts[4]) * original_height

                # Appliquer le redimensionnement
                x_center_resized = x_center * width_ratio
                y_center_resized = y_center * height_ratio
                width_resized = width * width_ratio
                height_resized = height * height_ratio

                # Normaliser les nouvelles coordonnées
                x_center_normalized = x_center_resized / new_size[0]
                y_center_normalized = y_center_resized / new_size[1]
                width_normalized = width_resized / new_size[0]
                height_normalized = height_resized / new_size[1]

                # Écrire la nouvelle annotation
                label_file.write(f"{class_id} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}\n")

# Exemple d'utilisation
image_dir = '/kaggle/working/renamed_dataset/images'
label_dir = '/kaggle/working//renamed_dataset/labels'
output_image_dir = '/kaggle/working/resized_dataset/images_resized'
output_label_dir = '/kaggle/working/resized_dataset/labels_resized'
new_size = (640, 640)  # Nouvelle taille souhaitée

# Créer les répertoires de sortie s'ils n'existent pas
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Parcourir les images et leurs annotations
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.jpg', '.png')):
        image_path = os.path.join(image_dir, image_name)
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)

        output_image_path = os.path.join(output_image_dir, image_name)
        output_label_path = os.path.join(output_label_dir, label_name)

        resize_image_and_update_labels(image_path, label_path, output_image_path, output_label_path, new_size)


In [ ]:
import os
import random
import shutil

def split_data(images_path, labels_path, output_path, train_ratio=0.8):
    """
    Split data into train and test sets from separate image and label directories.

    :param images_path: Path to the folder containing images.
    :param labels_path: Path to the folder containing labels.
    :param output_path: Path to the output folder where train and test splits will be saved.
    :param train_ratio: Ratio of data to include in the train set (default 0.8).
    """
    # Create main output directory
    os.makedirs(output_path, exist_ok=True)

    # Define output directories for train and test
    train_images_path = os.path.join(output_path, "train/images")
    train_labels_path = os.path.join(output_path, "train/labels")
    test_images_path = os.path.join(output_path, "test/images")
    test_labels_path = os.path.join(output_path, "test/labels")

    for path in [train_images_path, train_labels_path, test_images_path, test_labels_path]:
        os.makedirs(path, exist_ok=True)

    # Get list of all image and label files
    images = sorted(os.listdir(images_path))
    labels = sorted(os.listdir(labels_path))

    # Ensure images and labels are aligned
    if len(images) != len(labels):
        raise ValueError("Mismatch between number of images and labels")

    for img, lbl in zip(images, labels):
        if os.path.splitext(img)[0] != os.path.splitext(lbl)[0]:
            raise ValueError(f"Mismatch between image {img} and label {lbl}")

    # Shuffle and split data
    indices = list(range(len(images)))
    random.shuffle(indices)

    train_count = int(len(images) * train_ratio)
    train_indices = indices[:train_count]
    test_indices = indices[train_count:]

    # Move files to train and test folders
    for i in train_indices:
        shutil.copy(os.path.join(images_path, images[i]), train_images_path)
        shutil.copy(os.path.join(labels_path, labels[i]), train_labels_path)

    for i in test_indices:
        shutil.copy(os.path.join(images_path, images[i]), test_images_path)
        shutil.copy(os.path.join(labels_path, labels[i]), test_labels_path)

    print(f"Data split completed: {train_count} train, {len(images) - train_count} test")

# Paths
images_path = "/kaggle/working/resized_dataset/images_resized"
labels_path = "/kaggle/working/resized_dataset/labels_resized"
output_path = "/kaggle/working/splited_dataset"

# Call the function
split_data(images_path, labels_path, output_path, train_ratio=0.8)


In [ ]:
!pip install ultralytics

In [ ]:
from pathlib import Path
import yaml
import random

workingPath = "/kaggle/working/splited_dataset"

# train and test split based on percentage
trainPath = Path(workingPath).joinpath('train/images')
trainAllFiles = [str(file) for file in trainPath.glob('*') if str(file).endswith(('.jpg', '.png'))]
random.shuffle(trainAllFiles)

trainTestSplit = 0.8
partitionPoint = int(trainTestSplit * len(trainAllFiles))
trainFiles = trainAllFiles[:partitionPoint]
validationFiles = trainAllFiles[partitionPoint:]

# Save file lists
trainFile = Path(workingPath).joinpath('train.txt')
valFile = Path(workingPath).joinpath('val.txt')
with open(trainFile, "w") as train_file:
    train_file.write("\n".join(trainFiles))
with open(valFile, "w") as val_file:
    val_file.write("\n".join(validationFiles))
data = {
    'train': str(trainFile),
    'val': str(valFile),
    'test' : str(Path(workingPath).joinpath('test/images')),
    'nc': 60,
    'names': {
            0: "Aluminium foil",
            1: "Battery",
            2: "Aluminium blister pack",
            3: "Carded blister pack",
            4: "Other plastic bottle",
            5: "Clear plastic bottle",
            6: "Glass bottle",
            7: "Plastic bottle cap",
            8: "Metal bottle cap",
            9: "Broken glass",
            10: "Food Can",
            11: "Aerosol",
            12: "Drink can",
            13: "Toilet tube",
            14: "Other carton",
            15: "Egg carton",
            16: "Drink carton",
            17: "Corrugated carton",
            18: "Meal carton",
            19: "Pizza box",
            20: "Paper cup",
            21: "Disposable plastic cup",
            22: "Foam cup",
            23: "Glass cup",
            24: "Other plastic cup",
            25: "Food waste",
            26: "Glass jar",
            27: "Plastic lid",
            28: "Metal lid",
            29: "Other plastic",
            30: "Magazine paper",
            31: "Tissues",
            32: "Wrapping paper",
            33: "Normal paper",
            34: "Paper bag",
            35: "Plastified paper bag",
            36: "Plastic film",
            37: "Six pack rings",
            38: "Garbage bag",
            39: "Other plastic wrapper",
            40: "Single-use carrier bag",
            41: "Polypropylene bag",
            42: "Crisp packet",
            43: "Spread tub",
            44: "Tupperware",
            45: "Disposable food container",
            46: "Foam food container",
            47: "Other plastic container",
            48: "Plastic gloves",
            49: "Plastic utensils",
            50: "Pop tab",
            51: "Rope & strings",
            52: "Scrap metal",
            53: "Shoe",
            54: "Squeezable tube",
            55: "Plastic straw",
            56: "Paper straw",
            57: "Styrofoam piece",
            58: "Unlabeled litter",
            59: "Cigarette"
        }
}


# dump yaml file
yaml_output_path = Path(workingPath).joinpath('data.yaml')
with open(yaml_output_path, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

In [ ]:
from ultralytics import YOLO
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can replace 'yolov8n.pt' with other variants like 'yolov8s.pt'

# Train the model
model.train(
    data=yaml_output_path,      # Path to the YAML configuration file
    epochs=50,                # Number of epochs
    batch=16,                 # Batch size
    project='YOLOv8_Training',  # Project name
    name='trash_detection',  # Name for this training run
    save_period=1
)

# Save the best-trained model
trained_model_path = 'YOLOv8_Training/trash_detection/best.pt'
print(f"Training completed! Trained model saved at: {trained_model_path}")

In [ ]:
from ultralytics import YOLO

# Charger le modèle entraîné avec le fichier 'best.pt'
model = YOLO('YOLOv8_Training/trash_detection/weights/best.pt')

# Tester sur une image
results = model.predict(source='/kaggle/working/splited_dataset/test/images')  # Remplacez par le chemin réel de l'image

# Afficher les résultats
for result in results:
    result.show()


In [ ]:
model = YOLO('YOLOv8_Training/trash_detection/weights/best.pt')
model.export(format="onnx")